In [2]:
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Input
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten
from keras import backend as k
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization

2023-04-14 13:41:43.177651: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 13:41:43.546825: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-14 13:41:43.548850: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 13:41:44.970970: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)
img_rows, img_cols=28, 28

#reshaping
#this assumes our data format
#For 3D data, reshape 2D->3D
if k.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
#more reshaping
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

X_valid = X_valid.astype('float32')
X_valid /= 255

print('X_train shape:', X_train.shape)
print('X_valid shape:', X_valid.shape)
print('X_test shape:', X_test.shape)

X_train shape: (45000, 28, 28, 1)
X_valid shape: (15000, 28, 28)
X_test shape: (10000, 28, 28, 1)


In [4]:
#set number of categories
num_category = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)
y_valid = keras.utils.to_categorical(y_valid, num_category)

In [5]:
##model building
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))

model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(num_category, activation='softmax'))

2023-04-14 13:41:53.968311: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(),
              metrics=['accuracy'])

In [7]:
tf.keras.backend.clear_session() # run this to clear backend before recompiling model

In [8]:
batch_size = 128
num_epoch = 50
#model training
model_log = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/50
352/352 [==============================] - 46s 127ms/step - loss: 0.3111 - accuracy: 0.9192 - val_loss: 2.9547 - val_accuracy: 0.3183
Epoch 2/50
352/352 [==============================] - 44s 125ms/step - loss: 0.1292 - accuracy: 0.9645 - val_loss: 0.0661 - val_accuracy: 0.9794
Epoch 3/50
352/352 [==============================] - 46s 132ms/step - loss: 0.0890 - accuracy: 0.9742 - val_loss: 0.0531 - val_accuracy: 0.9829
Epoch 4/50
352/352 [==============================] - 43s 123ms/step - loss: 0.0671 - accuracy: 0.9806 - val_loss: 0.0506 - val_accuracy: 0.9856
Epoch 5/50
352/352 [==============================] - 43s 123ms/step - loss: 0.0633 - accuracy: 0.9808 - val_loss: 0.0417 - val_accuracy: 0.9888
Epoch 6/50
352/352 [==============================] - 44s 125ms/step - loss: 0.0534 - accuracy: 0.9834 - val_loss: 0.0453 - val_accuracy: 0.9865
Epoch 7/50
352/352 [==============================] - 46s 130ms/step - loss: 0.0457 - accuracy: 0.9860 - val_loss: 0.0455 - val_ac